In [1]:
import numpy as np
import rpy2.robjects as robj
import rpy2.robjects.vectors as rvec
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

from mrashpen.inference.mrash_wrapR import MrASHR
from mrashpen.inference.penalized_regression import PenalizedRegression as PLR

In [2]:
mrashR = importr('mr.ash.alpha')
glmnetR = importr('glmnet')

In [3]:
'''
get exact same X and y with the same seed as used by Peter.
https://github.com/pcarbo/mr-ash/blob/master/scripts/
'''

# Simulate data from the linear regression model y = x1*b1 + x2*b2 + e,
# e ~ N(0,1) with b1 = 0.1, b2 = 0.1.

n = 1000
seed = 1
robj.r('set.seed({})'.format(seed))
x0 = np.array(robj.r.rnorm(n))
X  = np.vstack((x0, x0)).T
beta = np.array([0.1, 0.1])
yerr = robj.r.rnorm(n)
y    = np.dot(X, beta) + np.array(yerr)

In [4]:
glmnet_fit = glmnetR.glmnet(X, y, alpha = 0.9, standardize = False)
b = np.array(robj.r['as.matrix'](robj.r.coef(glmnet_fit)))[1:]
print (np.max(np.abs(b[0,:] - b[1,:])))
print (glmnet_fit.rx2['npasses'])

0.08324812509587243
[244]


In [5]:
glmnet_fit = glmnetR.glmnet(X, y, alpha = 0.9, thres = 1e-15, standardize = False)
b = np.array(robj.r['as.matrix'](robj.r.coef(glmnet_fit)))[1:]
print (np.max(np.abs(b[0,:] - b[1,:])))
print (glmnet_fit.rx2['npasses'])

0.00022801184843943
[67400]


In [6]:
sk2 = np.array([0, 1, 2])
sk  = np.sqrt(sk2)
wk  = np.array([0, 0.5, 0.5])

mrash_r = MrASHR(option = "r2py", debug = True)
mrash_r.fit(X, y, sk, s2init = 1, winit = wk, update_pi = False, update_sigma2 = False, maxiter = 100)
print (mrash_r.coef)

Mr.ASH terminated at iteration 100.
[0.19223657 0.01418546]


In [7]:
mrash_r = MrASHR(option = "r2py", debug = True)
mrash_r.fit(X, y, sk, s2init = 1, winit = wk, update_pi = False, update_sigma2 = False, maxiter = 8000)
print (mrash_r.coef)

plr_lbfgs = PLR(method = 'L-BFGS-B', optimize_w = False, optimize_s = False, is_prior_scaled = True,
                debug = False, display_progress = False, calculate_elbo = False)
plr_lbfgs.fit(X, y, sk, binit = None, winit = wk, s2init = 1)
print(plr_lbfgs.coef)

Mr.ASH terminated at iteration 6139.
[0.10318978 0.10316636]
mr.ash.pen terminated at iteration 2.
[0.10316498 0.10316498]


In [8]:
mrash_r = MrASHR(option = "r2py", debug = True)
mrash_r.fit(X, y, sk, s2init = 1, winit = wk, update_pi = True, update_sigma2 = True, maxiter = 8000)
print (mrash_r.coef)

plr_lbfgs = PLR(method = 'L-BFGS-B', optimize_w = True, optimize_s = True, is_prior_scaled = True,
                debug = False, display_progress = False, calculate_elbo = False)
plr_lbfgs.fit(X, y, sk, binit = None, winit = wk, s2init = 1)
print(plr_lbfgs.coef)

Mr.ASH terminated at iteration 23.
[2.06233570e-01 6.26480391e-06]
mr.ash.pen terminated at iteration 28.
[0.10315504 0.10315504]
